In [1]:
# Lab 12 Character Sequence RNN
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # reproducibility

sample = " 30 40 50 60 70 80 90"
idx2char = list(set(sample))  # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex

# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
learning_rate = 0.1

sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

x_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0

cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]

        print(i, "loss:", l, "Prediction:", ''.join(result_str))

0 loss: 2.20759 Prediction: 00000000000000000000
1 loss: 1.99278 Prediction: 00000 00 0      0 00
2 loss: 1.75843 Prediction: 00000 00 00 00 00 00
3 loss: 1.58786 Prediction: 00 00 00 00 00 00 00
4 loss: 1.48224 Prediction: 00 00 00 00 00 00 00
5 loss: 1.34777 Prediction: 00 00 00 00 00 00 00
6 loss: 1.17432 Prediction: 00 00 00 00 00 00 00
7 loss: 1.0169 Prediction: 00 00 00 00 00 00 00
8 loss: 0.896938 Prediction: 00 00 60 60 60 60 60
9 loss: 0.815759 Prediction: 00 40 60 60 60 60 60
10 loss: 0.743901 Prediction: 00 40 90 60 60 60 60
11 loss: 0.689166 Prediction: 00 40 90 60 60 60 60
12 loss: 0.635082 Prediction: 00 40 50 60 60 60 60
13 loss: 0.593588 Prediction: 00 40 50 90 90 60 90
14 loss: 0.543011 Prediction: 30 40 50 90 90 80 90
15 loss: 0.500256 Prediction: 30 40 50 80 80 80 90
16 loss: 0.454425 Prediction: 30 40 40 80 80 80 90
17 loss: 0.413524 Prediction: 30 40 40 70 60 80 90
18 loss: 0.378151 Prediction: 30 40 40 80 80 80 90
19 loss: 0.333532 Prediction: 30 40 50 80 80 80 90

In [8]:
sample = " if you want you"
idx2char = list(set(sample))  # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex

print(idx2char)

print(char2idx)

print(len(char2idx))

sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello

print(sample_idx)
print(x_data)
print(y_data)

['a', 'y', 'i', 'u', 'o', 't', 'f', 'n', 'w', ' ']
{'f': 6, 'a': 0, 'n': 7, 'y': 1, 'w': 8, 'i': 2, 'u': 3, 'o': 4, 't': 5, ' ': 9}
10
[9, 2, 6, 9, 1, 4, 3, 9, 8, 0, 7, 5, 9, 1, 4, 3]
[[9, 2, 6, 9, 1, 4, 3, 9, 8, 0, 7, 5, 9, 1, 4]]
[[2, 6, 9, 1, 4, 3, 9, 8, 0, 7, 5, 9, 1, 4, 3]]
